In [1]:
import cv2
import numpy as np
from keras.models import load_model

In [2]:
# Load the trained model
model = load_model("emotion_detection_model.h5")

In [3]:
# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:
# Function to predict emotion on a face
def predict_emotion(face_roi):
    # Resize and normalize the face region
    resized_face = cv2.resize(face_roi, (48, 48)) / 255.0

    # Reshape for model input
    input_data = resized_face.reshape(1, 48, 48, 1)

    # Make prediction
    prediction = model.predict(input_data)

    # Get the predicted emotion
    emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
    predicted_emotion = emotion_labels[np.argmax(prediction)]

    return predicted_emotion


In [5]:
# Function to process video frames from camera
def process_video():
    # Open the camera
    cap = cv2.VideoCapture(0)

    # Check if the camera is opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    while True:
        # Read a frame from the camera
        ret, frame = cap.read()

        if not ret:
            print("Error: Could not read frame.")
            break

        # Convert the frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray_frame, 1.1, 4)

        # Iterate over detected faces
        for (x, y, w, h) in faces:
            # Extract the face region
            face_roi = gray_frame[y:y+h, x:x+w]

            # Predict emotion on the face
            emotion = predict_emotion(face_roi)

            # Draw bounding box around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Display the predicted emotion
            cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow("Facial Emotion Recognition", frame)

        # Check for key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

In [6]:
# Call the function to process video frames
process_video()

1/1 [==============================] - 0s 31ms/step
